<a href="https://colab.research.google.com/github/sunnysavita10/Indepth-GENAI/blob/main/RAG_With_Knowledge_graph(Neo4j).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# langchain-core

contains simple, core abstractions that have emerged as a standard, as well as LangChain Expression Language as a way to compose these components together. This package is now at version 0.1 and all breaking changes will be accompanied by a minor version bump.

# langchain-community
contains all third party integrations. We will work with partners on splitting key integrations out into standalone packages over the next month.

# langchain
contains higher-level and use-case specific chains, agents, and retrieval algorithms that are at the core of your application's cognitive architecture. We are targeting a launch of a stable 0.1 release for langchain in early January.#

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain_google_genai langchain-groq langchain-experimental langchain-neo4j neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.4/180.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
## Graphdb configuration(Download and set in env )
import os
from google.colab import userdata

# Fetch secrets from Colab Secrets storage
NEO4J_URI = userdata.get("NEO4J_URI")
NEO4J_USERNAME = userdata.get("NEO4J_USERNAME")
NEO4J_PASSWORD = userdata.get("NEO4J_PASSWORD")



In [3]:
# Graph
from langchain_neo4j import  Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [4]:
# LLM
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI

# GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
groq_api_key=userdata.get("groq_api_key")


# llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f3deee0ea50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f3dee907090>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
# Data
from langchain_community.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="NGC 4144").load()
raw_documents

[Document(metadata={'title': 'NGC 4144', 'summary': 'NGC 4144 is a barred spiral galaxy located in the Ursa Major constellation. It was discovered by the German-born British astronomer William Herschel on April 10, 1788. The galaxy is also known by several other catalog names, including IRAS F12074+4644, 2MASX J12095860+4627258, UGC 7151, and PGC 38688. It has an apparent magnitude of 12.17 in the blue band.', 'source': 'https://en.wikipedia.org/wiki/NGC_4144'}, page_content='NGC 4144 is a barred spiral galaxy located in the Ursa Major constellation. It was discovered by the German-born British astronomer William Herschel on April 10, 1788. The galaxy is also known by several other catalog names, including IRAS F12074+4644, 2MASX J12095860+4627258, UGC 7151, and PGC 38688. It has an apparent magnitude of 12.17 in the blue band.\n\n\n== Description ==\nNGC 4144 is a barred spiral galaxy classified as Im/BCD C, meaning it displays features of both irregular and blue compact dwarf galaxie

In [7]:
len(raw_documents)

19

In [8]:
# chunking
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)

In [9]:
# create graph
from langchain_experimental.graph_transformers.llm import LLMGraphTransformer #(Runs the query and generated the graph )
llm_transformer=LLMGraphTransformer(llm=llm)

In [10]:
# Convert the document to graph document
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [11]:
graph_documents # For proper visualizing(https://llm-graph-builder.neo4jlabs.com/)

[GraphDocument(nodes=[Node(id='Ngc 4144', type='Galaxy', properties={}), Node(id='William Herschel', type='Person', properties={}), Node(id='Ursa Major', type='Constellation', properties={}), Node(id='Iras F12074+4644', type='Catalog name', properties={}), Node(id='2Masx J12095860+4627258', type='Catalog name', properties={}), Node(id='Ugc 7151', type='Catalog name', properties={}), Node(id='Pgc 38688', type='Catalog name', properties={}), Node(id='Ursa Major Group', type='Galaxy group', properties={}), Node(id='Earth', type='Planet', properties={})], relationships=[Relationship(source=Node(id='William Herschel', type='Person', properties={}), target=Node(id='Ngc 4144', type='Galaxy', properties={}), type='DISCOVERED', properties={}), Relationship(source=Node(id='Ngc 4144', type='Galaxy', properties={}), target=Node(id='Ursa Major', type='Constellation', properties={}), type='LOCATED_IN', properties={}), Relationship(source=Node(id='Ngc 4144', type='Galaxy', properties={}), target=Node

In [12]:
graph_documents[0].nodes

[Node(id='Ngc 4144', type='Galaxy', properties={}),
 Node(id='William Herschel', type='Person', properties={}),
 Node(id='Ursa Major', type='Constellation', properties={}),
 Node(id='Iras F12074+4644', type='Catalog name', properties={}),
 Node(id='2Masx J12095860+4627258', type='Catalog name', properties={}),
 Node(id='Ugc 7151', type='Catalog name', properties={}),
 Node(id='Pgc 38688', type='Catalog name', properties={}),
 Node(id='Ursa Major Group', type='Galaxy group', properties={}),
 Node(id='Earth', type='Planet', properties={})]

In [13]:
graph_documents[0].relationships

[Relationship(source=Node(id='William Herschel', type='Person', properties={}), target=Node(id='Ngc 4144', type='Galaxy', properties={}), type='DISCOVERED', properties={}),
 Relationship(source=Node(id='Ngc 4144', type='Galaxy', properties={}), target=Node(id='Ursa Major', type='Constellation', properties={}), type='LOCATED_IN', properties={}),
 Relationship(source=Node(id='Ngc 4144', type='Galaxy', properties={}), target=Node(id='Iras F12074+4644', type='Catalog name', properties={}), type='ALSO_KNOWN_AS', properties={}),
 Relationship(source=Node(id='Ngc 4144', type='Galaxy', properties={}), target=Node(id='2Masx J12095860+4627258', type='Catalog name', properties={}), type='ALSO_KNOWN_AS', properties={}),
 Relationship(source=Node(id='Ngc 4144', type='Galaxy', properties={}), target=Node(id='Ugc 7151', type='Catalog name', properties={}), type='ALSO_KNOWN_AS', properties={}),
 Relationship(source=Node(id='Ngc 4144', type='Galaxy', properties={}), target=Node(id='Pgc 38688', type='Ca

In [14]:
# Store to graph db
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)


# For visualize in neo4j , query:-
MATCH (n)
OPTIONAL MATCH (n)-[r]->(m)
RETURN n, r, m

In [45]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

<ipython-input-45-55bffb0d4c21>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [47]:
vector_index = Neo4jVector.from_existing_graph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    embedding=embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)


In [48]:
vector_index

In [52]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


In [53]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [54]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [55]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [58]:
entity_chain.invoke({"question": "Where was William Herschel  born?"}).names

['William Herschel']

In [60]:
# Remove unnecessary characters
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [80]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
              """CALL db.index.fulltext.queryNodes('entity', $query, {limit: 2})
              YIELD node, score
              CALL {
                WITH node
                MATCH (node)-[r]->(neighbor)
                WHERE type(r) <> 'MENTIONS'
                RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
                UNION ALL
                WITH node
                MATCH (node)<-[r]-(neighbor)
                WHERE type(r) <> 'MENTIONS'
                RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
              }
              RETURN output LIMIT 50

            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [81]:
print(structured_retriever("Who is William Herschel ?"))

William Herschel - DISCOVERED -> Ngc 4144
William Herschel - DISCOVERED -> Ngc 4701
William Herschel - DISCOVERED -> Ngc 4041
Ngc 4041 - DISCOVERED_BY -> William Herschel


In [82]:
# The above information i will provide to llm , i will connect the simmilarity search along with structured retriver

def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question) # The above schema or structure(structured retriver)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)] # index  Similarity search
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [83]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [84]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [85]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [86]:
# kepp chat history
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [87]:
# dynamically pass anything during  run time
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [88]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | llm
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [89]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [90]:
prompt = ChatPromptTemplate.from_template(template)

In [91]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [92]:
chain.invoke({"question": "What is NGC 4144?"})

Search query: What is NGC 4144?


'NGC 4144 is a galaxy located in the Ursa Major Constellation, also known by other names such as IRAS F12074+4644, 2MASX J12095860+4627258, UGC 7151, and PGC 38688.'

In [93]:
chain.invoke({"question": "Who was William Herschel ?"})

Search query: Who was William Herschel ?


'William Herschel was a discoverer of various celestial objects, including Ngc 4144, Ngc 4701, and Ngc 4041.'

In [ ]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
from typing import Tuple, List, Optional

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain_core.runnables import ConfigurableField

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [ ]:
import os

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
from langchain_community.vectorstores import Neo4jVector

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [ ]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='Elizabeth I', type='Person'), Node(id='England', type='Country'), Node(id='Ireland', type='Country'), Node(id='House Of Tudor', type='Royal family'), Node(id='Henry Viii', type='Person'), Node(id='Anne Boleyn', type='Person'), Node(id='Edward Vi', type='Person'), Node(id='Lady Jane Grey', type='Person'), Node(id='Mary', type='Person'), Node(id='William Cecil', type='Person'), Node(id='Baron Burghley', type='Title'), Node(id='James Vi Of Scotland', type='Person'), Node(id='Mary, Queen Of Scots', type='Person'), Node(id='France', type='Country'), Node(id='Spain', type='Country'), Node(id='Sir Francis Walsingham', type='Person')], relationships=[Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='England', type='Country'), type='RULED'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Ireland', type='Country'), type='RULED'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='House Of 

In [ ]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [ ]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
from typing import Tuple, List, Optional

In [ ]:
from langchain_community.vectorstores import Neo4jVector

In [ ]:
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [ ]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
entity_chain.invoke({"question": "Where was Amelia Earhart born?"}).names

['Amelia Earhart']

In [ ]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [ ]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [ ]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
print(structured_retriever("Who is Elizabeth I?"))

Elizabeth I - RULED -> England
Elizabeth I - RULED -> Ireland
Elizabeth I - BELONGS_TO -> House Of Tudor
Elizabeth I - CHILD_OF -> Henry Viii
Elizabeth I - CHILD_OF -> Anne Boleyn
Elizabeth I - HALF-SISTER_OF -> Edward Vi
Elizabeth I - HALF-SISTER_OF -> Mary
Elizabeth I - IGNORED_CLAIMS_OF -> Lady Jane Grey
Elizabeth I - DEPENDS_ON -> William Cecil
Elizabeth I - DEPENDS_ON -> Sir Francis Walsingham
Elizabeth I - CREATED_TITLE -> Baron Burghley
Elizabeth I - SUCCEEDED_BY -> James Vi Of Scotland
Elizabeth I - MOTHER_OF -> Mary, Queen Of Scots
Elizabeth I - MANOEUVRING_BETWEEN -> France
Elizabeth I - MANOEUVRING_BETWEEN -> Spain
Elizabeth I - SUCCESSOR -> James Vi Of Scotland
Henry Viii - SUCCESSOR -> Elizabeth I


In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [ ]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [ ]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [ ]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke({"question": "Which house did Elizabeth I belong to?"})

Search query: Which house did Elizabeth I belong to?


'Elizabeth I belonged to the House of Tudor.'

In [ ]:
chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)

Search query: When was Elizabeth I born?


'Elizabeth I was born on 7 September 1533.'